# Simple Notebook for Visualizing Accelerometer Data  of 10 Steps

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import butter,filtfilt,find_peaks
import plotly.graph_objects as go


## Read the Data

In [34]:
df_acc = pd.read_csv('../Data/Data_Nils/10_schritte/accelerometer.csv')
readings_np = df_acc.to_numpy()
readings_np.shape

(4845, 5)

## Calculate Norm of the Accelerometer Data

In [35]:
def norm(data):
    temp = 0
    for i in data:
        temp += (i*i)
    return np.sqrt(temp)

norm_readings_np = np.apply_along_axis(norm, 1, readings_np[:, 1:-1])

## Implement a Butterworthfilter Helper Function

In [36]:
def butter_lowpass_filter(data):
    cutoff = 2
    order = 8
    normal_cutoff = cutoff / 100
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)
    y = filtfilt(b, a, data)
    return y

### Filter All Readings Besides the Time

In [37]:
filtered_readings_np = np.apply_along_axis(butter_lowpass_filter, 0, norm_readings_np)

In [38]:
filtered_readings_np = np.c_[readings_np[:, 0], filtered_readings_np]

## Plot the Filtered Data

In [39]:
# Filter the data, and plot both the original and filtered signals.
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 1],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )

fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 0],
            line =  dict(shape =  'spline' ),
            name = 'time signal')
            )
fig.show()

## Implement a Helper for the Peak Finder Function

In [45]:
def helper_find_peaks(data):
    ret, _ = find_peaks(data, height=0.75)
    return ret

## Find the peaks

In [46]:
# find peaks in absolute acceleration -> resulting in steps
peaks = []


peaks.append(helper_find_peaks(filtered_readings_np[:, 1]))

peaks

[array([1955, 2107, 2264, 2426, 2604, 2721, 2887, 3020, 3166, 3318],
       dtype=int64)]

## Show Peaks In Z-Axis

In [47]:
filtered_df = pd.DataFrame(filtered_readings_np, columns=['time', 'norm'])

In [48]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_df['norm'],
            line =  dict(shape =  'spline' ),
            name = 'normalized signal')
            )
fig.add_trace(go.Scatter(
            x = peaks[0],
            y = filtered_df.loc[peaks[0]]['norm'],
            mode =  'markers',
            name = 'peaks')
            )